In [2]:
import pandas as pd
import re

import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from matplotlib import pyplot as plt

%matplotlib inline
import seaborn as sns
import eli5
import numpy as np
import nltk

nltk.download("stopwords")
from nltk.corpus import stopwords

russian_stopwords = stopwords.words("russian")

[nltk_data] Downloading package stopwords to /home/andrey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<IPython.core.display.Javascript object>

### Модуль с загрузкой документа и извлечением необходимых данных

In [3]:
import os
import docx
import re
import pickle

<IPython.core.display.Javascript object>

In [4]:
def get_text(doc):
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text.strip())
    fullText = [elem for elem in fullText if len(elem) > 0]
    return fullText

<IPython.core.display.Javascript object>

In [5]:
def get_first_index(fullText):
    ind_start = 0
    for i in range(len(fullText) - 1):
        if ("правила" in fullText[i].lower()) and (
            "предоставления" in fullText[i + 1].lower()
        ):
            ind_start = i
            break
    return ind_start

<IPython.core.display.Javascript object>

In [6]:
def get_end_index(fullText):
    end_start = len(fullText)
    for i in range(ind_start, len(fullText) - 1):
        if (
            (
                fullText[i].lower().startswith("утвержден")
                & fullText[i + 1].lower().startswith("постановлением правительства")
            )
            | (
                ("приложение №" in fullText[i].lower())
                & ("предоставления" in fullText[i + 1].lower())
            )
            | (
                ("приложение №" in fullText[i].lower())
                & ("предоставления" in fullText[i].lower())
            )
            | (
                ("приложение N" in fullText[i].lower())
                & ("предоставления" in fullText[i + 1].lower())
            )
            | (
                ("приложение" in fullText[i].lower())
                & ("к правилам предоставления" in fullText[i + 1].lower())
            )
        ):
            if (i < end_start - 6) & ("правила" in fullText[i + 4].lower()) and (
                "предоставления" in fullText[i + 5].lower()
            ):
                continue
            end_start = i
            break
    return end_start

<IPython.core.display.Javascript object>

In [7]:
path_docs = "../data/test_real/"

<IPython.core.display.Javascript object>

In [8]:
interest_files = [file for file in os.listdir(path_docs) if file.endswith("docx")]
sorted(interest_files)

['1.docx',
 '10.docx',
 '2.docx',
 '3.docx',
 '4.docx',
 '5.docx',
 '6.docx',
 '7.docx',
 '8.docx',
 '9.docx']

<IPython.core.display.Javascript object>

In [9]:
sample_sub = pd.read_csv("../data/test_real/description/Название_команды.csv", sep=";")
sample_sub

,file_id,id,class
0,1,1,1
1,1,2,1
2,1,3,1
3,1,4,1
4,1,5,1
...,...,...,...
847,10,67,1
848,10,68,1
849,10,69,1
850,10,70,1


<IPython.core.display.Javascript object>

In [10]:
sample_sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 852 entries, 0 to 851
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   file_id  852 non-null    int64
 1   id       852 non-null    int64
 2   class    852 non-null    int64
dtypes: int64(3)
memory usage: 20.1 KB


<IPython.core.display.Javascript object>

### определяем текущую модель

In [11]:
from transformers import AutoModel, AutoTokenizer, BertForSequenceClassification
import torch
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# from keras.preprocessing.sequence import pad_sequences
from keras_preprocessing.sequence import pad_sequences
from tqdm.notebook import tqdm

device = "cpu"

<IPython.core.display.Javascript object>

In [12]:
# Define the model repo
model_name = "DeepPavlov/rubert-base-cased"


# Download pytorch model
tokenizer = AutoTokenizer.from_pretrained(model_name)

<IPython.core.display.Javascript object>

In [13]:
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=40)
PATH = "../models/epoch_new_bert_dict_4.ckpt"
model.load_state_dict(torch.load(PATH, map_location=torch.device("cpu")))
model.eval()

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were n

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

<IPython.core.display.Javascript object>

In [14]:
def prepare_data_bert(ref_text, tokenizer):
    tokenized_texts = [tokenizer.tokenize(sent) for sent in ref_text]
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

    input_ids = pad_sequences(
        input_ids, maxlen=256, dtype="long", truncating="post", padding="post"
    )
    attention_masks = [[float(i > 0) for i in seq] for seq in input_ids]

    prediction_inputs = torch.tensor(input_ids)
    prediction_masks = torch.tensor(attention_masks)
    # заглушка
    prediction_labels = torch.tensor([[0] for _ in range(len(ref_text))])
    prediction_data = TensorDataset(
        prediction_inputs, prediction_masks, prediction_labels
    )

    prediction_dataloader = DataLoader(
        prediction_data, sampler=SequentialSampler(prediction_data), batch_size=32
    )
    return prediction_dataloader

<IPython.core.display.Javascript object>

In [15]:
def bert_torch_inference(model, prediction_dataloader):
    model.eval()
    test_preds, test_labels = [], []
    test_probas = []
    for batch in tqdm(prediction_dataloader):
        # добавляем батч для вычисления на GPU
        batch = tuple(t.to(device) for t in batch)

        # Распаковываем данные из dataloader
        b_input_ids, b_input_mask, b_labels = batch

        # При использовании .no_grad() модель не будет считать и хранить градиенты.
        # Это ускорит процесс предсказания меток для тестовых данных.
        with torch.no_grad():
            logits = model(
                b_input_ids, token_type_ids=None, attention_mask=b_input_mask
            )
        probabilities = F.softmax(logits[0].detach(), dim=-1).cpu().numpy()
        probabilities = np.max(probabilities, axis=1)
        # Перемещаем logits и метки классов на CPU для дальнейшей работы
        logits = logits[0].detach().cpu().numpy()
        label_ids = b_labels.to("cpu").numpy()

        # Сохраняем предсказанные классы и ground truth
        batch_preds = np.argmax(logits, axis=1)
        batch_labels = np.concatenate(label_ids)
        test_preds.extend(batch_preds)
        test_labels.extend(batch_labels)
        test_probas.extend(probabilities)

    y_pred = test_preds
    y_proba = test_probas
    return y_pred, y_proba

<IPython.core.display.Javascript object>

### поехали

In [16]:
df_submission = []
for cur_doc in tqdm(sorted(interest_files)):
    filename = path_docs + cur_doc
    file_id = int(cur_doc.split(".")[0])
    doc = docx.Document(filename)
    fullText = get_text(doc)
    # сегментируем, где у нас потенциально есть классы
    ind_start = get_first_index(fullText)
    end_start = get_end_index(fullText)

    target_data = []
    activate_flag = False
    real_start_real = ind_start
    for num_el, elem in enumerate(fullText[ind_start:end_start]):
        if not activate_flag:
            if len(re.findall("\d+\. ", elem)) > 0:
                real_start_real += num_el
                activate_flag = True
                target_data.append(elem.replace("\xa0", " "))
        else:
            target_data.append(elem.replace("\xa0", " "))
    # извлекаем текст и id
    texts = []
    ids = []
    for num, elem in enumerate(target_data):
        if len(re.findall("{\d+", elem)) > 0:
            splits = re.split("\d+}", elem)
            cur_ids = re.findall("\d+}", elem)
            for i in range(1, len(splits), 2):
                texts.append(splits[i])
            for i in range(1, len(cur_ids), 2):
                ids.append(int(re.findall("\d+", cur_ids[i])[0]))

    df1 = pd.DataFrame({"texts": texts, "id": ids})

    # работа для модели
    ref_text = [re.sub("[\W\d]+", " ", x.lower()) for x in df1["texts"]]
    prediction_dataloader = prepare_data_bert(ref_text, tokenizer)
    y_pred, y_proba = bert_torch_inference(model, prediction_dataloader)
    #     y_proba = model.predict_proba(X_text)
    df1["class"] = y_pred
    df1["file_id"] = file_id
    df_submission.append(df1)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

In [17]:
df_submission = pd.concat(df_submission)

<IPython.core.display.Javascript object>

In [18]:
df_submission = df_submission[["file_id", "id", "class", "texts"]]

<IPython.core.display.Javascript object>

In [19]:
df_submission.sort_values(["file_id", "id"], inplace=True)

<IPython.core.display.Javascript object>

In [20]:
df_submission.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 852 entries, 0 to 70
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   file_id  852 non-null    int64 
 1   id       852 non-null    int64 
 2   class    852 non-null    int64 
 3   texts    852 non-null    object
dtypes: int64(3), object(1)
memory usage: 33.3+ KB


<IPython.core.display.Javascript object>

In [21]:
sample_sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 852 entries, 0 to 851
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   file_id  852 non-null    int64
 1   id       852 non-null    int64
 2   class    852 non-null    int64
dtypes: int64(3)
memory usage: 20.1 KB


<IPython.core.display.Javascript object>

In [22]:
df_submission[sample_sub.columns].to_csv("SibDS.csv", sep=";", index=False)

<IPython.core.display.Javascript object>

In [24]:
df_submission[df_submission["class"] == 0]

,file_id,id,class,texts
8,1,9,0,аренда помещений и оборудования;{
61,1,62,0,Основаниями для отказа в предоставлении субси...
2,3,3,0,"и государственной Российской Федерации ""Научн..."
38,3,39,0,"порядок отклонения заявок, а также информацию ..."
55,3,56,0,"документы, подтверждающие полномочия лица, дей..."
66,3,67,0,представивших заявки. {
69,3,70,0,По результатам рассмотрения указанных предложе...
100,3,101,0,"несоответствие участника конкурса требованиям,..."
108,3,109,0,размер субсидии и условия ее предоставления; {
127,3,128,0,nmax - общее количество работников образовател...


<IPython.core.display.Javascript object>